# Reconhecimento de Entidades Nomeadas com LSTM



### 1. Instalação e Importação de Bibliotecas

In [ ]:
# Instalação das bibliotecas necessárias
# %%capture evita que as saídas da instalação poluam o notebook, mantendo-o limpo.
%%capture
%pip install numpy pandas tensorflow scikit-learn

In [ ]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### 2. Carregamento e Leitura do Dataset

In [ ]:
# Carrega o dataset a partir de um arquivo CSV.
dados_brutos = pd.read_csv("data/ner_dataset.csv", encoding="latin1").ffill()
# Dataset utilizado: GMB (Groningen Meaning Bank)
# Fonte: Kaggle - https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus

### 3. Pré-processamento dos Dados

In [ ]:
# Agrupamento das palavras por sentença
sentencas = []
sentenca_atual = []
for _, linha in dados_brutos.iterrows():
    palavra, etiqueta = linha["Word"], linha["Tag"]
    if palavra == ".":
        sentenca_atual.append((palavra, etiqueta))
        sentencas.append(sentenca_atual)
        sentenca_atual = []
    else:
        sentenca_atual.append((palavra, etiqueta))

### 4. Criação dos Vocabulários

In [ ]:
# Criação de dicionários para conversão entre palavras/tags e seus respectivos índices
palavras = list(set(dados_brutos["Word"].values))
palavras.append("PADword")
etiquetas = list(set(dados_brutos["Tag"].values))

mapa_palavra_para_indice = {w: i + 1 for i, w in enumerate(palavras)}
mapa_indice_para_palavra = {i + 1: w for i, w in enumerate(palavras)}

mapa_tag_para_indice = {t: i for i, t in enumerate(etiquetas)}
mapa_indice_para_tag = {i: t for t, i in mapa_tag_para_indice.items()}

### 5. Preparação das Sequências para Treinamento

In [ ]:
# Transformação das sentenças e etiquetas em sequências de índices
max_len = 50

X = [[mapa_palavra_para_indice[palavra] for palavra, etiqueta in s] for s in sentencas]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=mapa_palavra_para_indice["PADword"])

y = [[mapa_tag_para_indice[etiqueta] for palavra, etiqueta in s] for s in sentencas]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=mapa_tag_para_indice["O"])
y = [to_categorical(i, num_classes=len(etiquetas)) for i in y]


### 6. Separação dos Dados em Treinamento e Teste

In [ ]:
# Divisão dos dados em subconjuntos de treinamento e teste com 10% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1)

### 7. Definição e Construção da Arquitetura do Modelo

In [ ]:
# Estrutura de rede recorrente bidirecional com camada de saída TimeDistributed
modelo = Sequential()
modelo.add(Embedding(input_dim=len(palavras) + 1, output_dim=64))
modelo.add(Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1)))
modelo.add(TimeDistributed(Dense(len(etiquetas), activation="softmax")))

# Compilação do modelo com otimizador Adam e função de perda categórica
modelo.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
modelo.summary()

### 8. Treinamento do Modelo

In [ ]:
# Treinamento com 5 épocas e validação em 10% dos dados
historico = modelo.fit(X_treino, np.array(y_treino), batch_size=32, epochs=5, validation_split=0.1, verbose=1)

### 9. Avaliação da Performance

In [ ]:
# Avaliação do modelo com base no conjunto de testes
resultado = modelo.evaluate(X_teste, np.array(y_teste))
print(f"Acurácia obtida no teste: {resultado[1]*100:.2f}%")

### 10. Testes com Predições

In [ ]:
# Exemplo de predição e reconstrução da sentença com suas respectivas entidades nomeadas
indice_exemplo = 123
predicao = modelo.predict(np.array([X_teste[indice_exemplo]]))
predicao_indices = np.argmax(predicao, axis=-1)

print("PALAVRA - VERDADEIRO - PREDITO")
for p, v in zip(X_teste[indice_exemplo], predicao_indices[0]):
    if p != mapa_palavra_para_indice["PADword"]:
        print(f"{mapa_indice_para_palavra[p]:15} - {mapa_indice_para_tag[np.argmax(y_teste[indice_exemplo][X_teste[indice_exemplo].tolist().index(p)])]} - {mapa_indice_para_tag[v]}")